In [32]:
import reverse_geocoder as rg  # Librería para realizar la geocodificación inversa
import pandas as pd           # Librería para manejar DataFrame

#importación de src codigo de soporte con las funciones necesarias para la limpieza
import sys
sys.path.append("..")
from src import sp_geocode as sp
from src import sp_eda_limpieza


In [33]:
#ajustes de visualización
pd.set_option('display.max_columns', None)

In [34]:
df_geocode_ba=pd.read_csv(r"C:\Users\matis\Desktop\EDA_MK_POWER\data\eda_limpieza_files\eda_mk_ok.csv",usecols=lambda col:not col.startswith('Unnamed'))

In [35]:
df_geocode_ba.columns

Index(['Age', 'Job', 'Marital', 'Education', 'Default', 'Housing', 'Loan',
       'Contact', 'Duration', 'Campaign', 'Previous', 'Poutcome',
       'Emp.var.rate', 'Cons.price.idx', 'Cons.conf.idx', 'Euribor3m',
       'Nr.employed', 'Subscribed', 'Date', 'Latitude', 'Longitude', 'ID',
       'Income', 'Kidhome', 'Teenhome', 'Dt_Customer', 'NumWebVisitsMonth'],
      dtype='object')

In [36]:
prueba_coordenadas = [(43.238, -79.899), (40.7128, -74.0060)]
try:
    resultados = rg.search(prueba_coordenadas)
    print(resultados)
except Exception as e:
    print(f"Error en reverse_geocoder: {e}")


[{'lat': '43.25011', 'lon': '-79.84963', 'name': 'Hamilton', 'admin1': 'Ontario', 'admin2': '', 'cc': 'CA'}, {'lat': '40.71427', 'lon': '-74.00597', 'name': 'New York City', 'admin1': 'New York', 'admin2': '', 'cc': 'US'}]


In [37]:
def geolocalizar(df, latcol, loncol, batch_size=100):
    # Crear lista de coordenadas y resultados
    coordenadas_lotes = []
    resultados = []

    # Iterar sobre el DataFrame fila por fila
    for index, row in df.iterrows():
        lat = row.get(latcol, None)
        lon = row.get(loncol, None)

        # Validar coordenadas: si falta una, asignar resultado nulo
        if pd.isna(lat) or pd.isna(lon):
            resultados.append((None, None, None))
        else:
            coordenadas_lotes.append((lat, lon))

    # Procesar en lotes las coordenadas válidas
    for i in range(0, len(coordenadas_lotes), batch_size):
        lote = coordenadas_lotes[i:i + batch_size]
        try:
            lote_resultados = rg.search(lote)
            resultados.extend([(r.get('name', None), r.get('cc', None), r.get('admin1', None)) for r in lote_resultados])
        except Exception as e:
            print(f"Error al procesar lote {lote}: {e}")
            resultados.extend([(None, None, None)] * len(lote))  # Rellenar con None en caso de error

    # Agregar resultados al DataFrame original
    ciudades, paises, regiones = zip(*resultados)
    df['City'] = ciudades
    df['Country'] = paises
    df['Region'] = regiones

    return df

# Ejemplo de prueba con un DataFrame
data = {
    'latitude': [43.238, 40.7128, 48.8566, -33.8688, 91.0],  # Incluye un valor fuera de rango
    'longitude': [-79.899, -74.0060, 2.3522, 151.2093, 50.0]
}
df = pd.DataFrame(data)

# Aplicar la función
df_resultado = geolocalizar(df, 'latitude', 'longitude', batch_size=2)

# Mostrar el resultado
print(df_resultado)

   latitude  longitude           City Country                      Region
0   43.2380   -79.8990       Hamilton      CA                     Ontario
1   40.7128   -74.0060  New York City      US                    New York
2   48.8566     2.3522          Paris      FR               Ile-de-France
3  -33.8688   151.2093         Sydney      AU             New South Wales
4   91.0000    50.0000      Iskateley      RU  Nenetskiy Avtonomnyy Okrug


In [38]:
df_geocoded=sp.geolocalizar(df_geocode_ba,'Latitude','Longitude')

In [39]:
df_geocoded.columns

Index(['Age', 'Job', 'Marital', 'Education', 'Default', 'Housing', 'Loan',
       'Contact', 'Duration', 'Campaign', 'Previous', 'Poutcome',
       'Emp.var.rate', 'Cons.price.idx', 'Cons.conf.idx', 'Euribor3m',
       'Nr.employed', 'Subscribed', 'Date', 'Latitude', 'Longitude', 'ID',
       'Income', 'Kidhome', 'Teenhome', 'Dt_Customer', 'NumWebVisitsMonth',
       'City', 'Country', 'Region'],
      dtype='object')

In [40]:
sp_eda_limpieza.eda_preliminar(df_geocoded)

,Age,Job,Marital,Education,Default,Housing,Loan,Contact,Duration,Campaign,Previous,Poutcome,Emp.var.rate,Cons.price.idx,Cons.conf.idx,Euribor3m,Nr.employed,Subscribed,Date,Latitude,Longitude,ID,Income,Kidhome,Teenhome,Dt_Customer,NumWebVisitsMonth,City,Country,Region
37961,33.0,services,married,basic_4y,no,no,no,telephone,152.0,1.0,0.0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,2019-01-12,45.888,-86.047,e168f8a4-2378-422b-a89d-11b83172f997,36998,0,0,2012-01-04,8,Walpole Island,CA,Ontario
15570,46.0,entrepreneur,married,basic_9y,no,no,no,cellular,169.0,1.0,0.0,nonexistent,1.4,93.444,-36.1,4.966,5228.0,no,2015-07-09,30.616,-124.554,5c3c08c2-3b0c-4111-b406-bc1668b81dd8,35200,2,1,2012-03-27,11,Sahuaripa,MX,Sonora
34047,52.0,admin.,divorced,basic_9y,no,yes,yes,cellular,9.0,5.0,1.0,failure,-1.8,93.075,-47.1,1.405,5099.0,no,2017-01-05,42.182,-98.153,c9ef2b43-8a59-4a82-8fdc-94405bf132a6,41771,2,1,2014-10-29,19,Huxley,US,Iowa
35109,51.0,retired,married,university_degree,NaN,no,no,cellular,94.0,3.0,0.0,nonexistent,1.4,93.918,-42.7,4.961,5228.0,no,2018-05-06,30.088,-122.020,d03face9-ec62-4d1a-8f1e-960ad12dca54,21478,1,1,2014-04-24,10,Alamo,US,Nevada
23506,36.0,blue_collar,divorced,high_school,no,no,no,telephone,531.0,2.0,0.0,nonexistent,1.1,93.994,-36.4,4.856,5191.0,no,2015-08-31,25.687,-76.940,8b275c6f-23a4-4846-a761-f6e135b28952,100910,2,0,2012-03-17,24,Talihina,US,Oklahoma


--------------------
INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43170 entries, 0 to 43169
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Age                37880 non-null  float64
 1   Job                42655 non-null  object 
 2   Marital            42915 non-null  object 
 3   Education          41193 non-null  object 
 4   Default            34019 non-null  object 
 5   Housing            41974 non-null  object 
 6   Loan               41974 non-null  object 
 7   Contact            43000 non-null  object 
 8   Duration           43000 non-null  float64
 9   Campaign           43000 non-null  float64
 10  Previous           43000 non-null  float64
 11  Poutcome           43000 non-null  object 
 12  Emp.var.rate       43000 non-null  float64
 13  Cons.price.idx     42529 non-null  float64
 14  Cons.conf.idx      43000 non-null  float64
 15  Euribor3m          33744 non-null  float64
 

None

--------------------
NULOS


Age                  12.25
Job                   1.19
Marital               0.59
Education             4.58
Default              21.20
Housing               2.77
Loan                  2.77
Contact               0.39
Duration              0.39
Campaign              0.39
Previous              0.39
Poutcome              0.39
Emp.var.rate          0.39
Cons.price.idx        1.48
Cons.conf.idx         0.39
Euribor3m            21.83
Nr.employed           0.39
Subscribed            0.39
Date                  0.97
Latitude              0.39
Longitude             0.39
ID                    0.00
Income                0.00
Kidhome               0.00
Teenhome              0.00
Dt_Customer           0.00
NumWebVisitsMonth     0.00
City                  0.39
Country               0.39
Region                0.39
dtype: float64

...

Country
US    32237
MX     4783
CA     3147
BS     1552
BM      909
TC      277
CU       95
Name: count, dtype: int64

--------------------


Region
California        2954
Texas             2559
North Carolina    1745
Ontario           1512
Montana           1483
                  ... 
Harbour Island      18
Freeport            15
Spanish Wells       15
Black Point          7
Artemisa             4
Name: count, Length: 88, dtype: int64

--------------------
VALORES NUMÉRICOS


,count,mean,std,min,25%,50%,75%,max
Age,37880.0,39.977112,10.437957,17.000,32.00000,38.0000,47.00000,98.000
Duration,43000.0,257.739279,258.666033,0.000,102.00000,179.0000,319.00000,4918.000
Campaign,43000.0,2.567233,2.772294,1.000,1.00000,2.0000,3.00000,56.000
Previous,43000.0,0.174023,0.497366,0.000,0.00000,0.0000,0.00000,7.000
Emp.var.rate,43000.0,0.077128,1.573898,-3.400,-1.80000,1.1000,1.40000,1.400
Cons.price.idx,42529.0,93.574219,0.579548,92.201,93.07500,93.7490,93.99400,94.767
Cons.conf.idx,43000.0,-40.509049,4.637186,-50.800,-42.70000,-41.8000,-36.40000,-26.900
Euribor3m,33744.0,3.616521,1.737117,0.634,1.34400,4.8570,4.96100,5.045
Nr.employed,43000.0,5166.835721,72.337206,4964.000,5099.00000,5191.0000,5228.00000,5228.000
Latitude,43000.0,36.856697,7.225948,24.396,30.61475,36.7610,43.11325,49.384


In [41]:
df_geocode_ba.to_csv('bankadditional_geocoded.csv')